In [1]:
import csv
 
with open('meteoreeltime.csv', 'w') as csvfile:
    fieldnames = ['numero','pression','direction_de_vent','vitesse_de_vent','temperature',  'humidité','visibilité', 'nebulosite','hauteur_neige', 'precipitations_1','precipitations_3','nom','latitude',  'longitude','timestamp']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    

In [3]:
import json
import requests
import time
import pandas as pd
import csv
from datetime import datetime
from starbase import Connection
c = Connection("192.168.122.206",8086)
t=c.table('meteo')
t.columns()

#while True:
        #tab ville is an array that contains almost every city that have flights comming from or going to lyon's airport
ville=["lyon","alger","annaba","batna","biskra","bejaia","constantine","oran","setif","tlemcen","londres","Agadir","casablanca","fes","essaouira","marrakech","tanger","djerba","monastir","tozeur","tunis","istanbul","berlin","düsseldorf","frankfurt","munich","vienne","bruxelles","copenhagen","alicante","barcelona","bilbao","madrid","malaga","seville","valence","helsinki","ajaccio","bastia","biarritz","bordeaux","brest","caen","figari","la rochelle","lille","limoges","lorient","marseille","metz","nancy","nantes","nice","paris","pau","poitiers","rennes","strasbourg","toulouse","budapest","athena","belfast","dublin","malta","bari","bologne","catane","milan","naples","rome","venise","amsterdam","krakow","warsaw","faro","lisbonne","porto","bucharest","cluj-napoca","moscou","zurich","prague","Göteborg","amman","dubai","beirut","montreal"]
lastdata=[]
for i in range(len(ville)):
            url="http://api.openweathermap.org/data/2.5/weather?q="+ville[i]+"&APPID=f2c147486ddbf152d20dee5ad2186656"
            #print(url)
            response = requests.get(url)
            data=response.json()
            data_clean={}
            #print(data)
            data_clean['numero']=120000
            data_clean['pression']=data['main']['pressure']*100
            data_clean['direction_de_vent']=0
            data_clean['vitesse_de_vent']=0
            data_clean['temperature']=data['main']['temp']
            data_clean['humidité']=data['main']['humidity']
            data_clean['nebulosite']=0
            data_clean['visibilité']=0
            data_clean['hauteur_neige']=0
            data_clean['precipitations_1']=0
            data_clean['precipitations_3']=0
            data_clean['nom']=data['name']
            data_clean['longitude']=data['coord']['lon']
            data_clean['latitude']=data['coord']['lat']
            data_clean['timestamp']=data['dt']

            if 'visibility' in data.keys():
                  data_clean['visibilité']=data['visibility']

            if 'wind' in data.keys():
                   if 'deg' in data['wind'].keys():
                       data_clean['direction_de_vent']=data['wind']['deg']
                   if 'speed' in data['wind'].keys():
                       data_clean['vitesse_de_vent']=data['wind']['speed']

            if 'rain' in data.keys():
             if 'rain.1h' in data['rain'].keys():
                 data_clean['precipitations_1']=data['rain']['rain.1h']
             if 'rain.3h' in data['rain'].keys():
                 data_clean['precipitations_3']=data['rain']['rain.3h']

            if 'snow' in data.keys():
             if 'snow.1h' in data['snow']:
                data_clean['hauteur_neige']=data['snow']['snow.1h']

            if 'clouds' in data.keys():
                data_clean['nebulosite']=data['clouds']['all']
            #ici on enregistre les donnees dans un fichier csv
            with open('meteoreeltime.csv', 'a') as csvfile:
                fieldnames = ['numero','pression','direction_de_vent','vitesse_de_vent','temperature',  'humidité','visibilité', 'nebulosite','hauteur_neige', 'precipitations_1','precipitations_3','nom','latitude',  'longitude','timestamp']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(data_clean)

            timekey = datetime.fromtimestamp(data_clean['timestamp']).strftime("%Y-%m-%dT:%H:%M:%S")
            mykey=data_clean['nom']+'_'+timekey

            t.insert(mykey,
                 {'meteo_info': {'numero':data_clean['numero'],
              'pression':data_clean['pression'],
              'direction_de_vent':data_clean['direction_de_vent'],
              'vitesse_de_vent': data_clean['vitesse_de_vent'],
              'temperature':data_clean['temperature'],
               'humidité':data_clean['humidité'],
              'visibilité': data_clean['visibilité'], 
              'nebulosite': data_clean['nebulosite'],
              'hauteur_neige':data_clean['hauteur_neige'],
              'precipitations_1': data_clean['precipitations_1'],
              'precipitations_3':data_clean['precipitations_3'],
               'nom':data_clean['nom'],
              'latitude': data_clean['longitude'],
               'longitude':data_clean['latitude'],
              'timestamp': data_clean['timestamp']}}
             )
    #every 5 hours
    #time.sleep(18000)



     
#lastdata.append(data_clean)


ModuleNotFoundError: No module named 'starbase'

In [23]:
from datetime import datetime
from starbase import Connection
c = Connection("192.168.122.206",8086)
t=c.table('meteo')
timekey = datetime.fromtimestamp(1578054142).strftime("%Y-%m-%dT:%H:%M:%S")

t.fetch('Lyon_'+timekey)


In [60]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("meteoreeltime.csv") 
len(df.index)
df['timestamp'][0]
for i in range(len(df.index)):
     df.loc[i,'rr'] =datetime.fromtimestamp( df['timestamp'][i]).strftime("%Y-%m-%d %H")
        

In [66]:
#df.isnull().sum()
df[df['nom']=='Marseille']


,numero,pression,direction_de_vent,vitesse_de_vent,temperature,humidité,visibilité,nebulosite,hauteur_neige,precipitations_1,precipitations_3,nom,latitude,longitude,timestamp,rr
48,120000,102900,290,1.5,286.02,81,10000,75,0.0,0.0,0.0,Marseille,43.3,5.37,1578053982,2020-01-03 12
135,120000,102900,290,1.5,286.02,81,10000,75,0.0,0.0,0.0,Marseille,43.3,5.37,1578053797,2020-01-03 12
222,120000,102900,260,1.5,286.02,76,10000,75,0.0,0.0,0.0,Marseille,43.3,5.37,1578054725,2020-01-03 12


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 15 columns):
numero               870 non-null int64
pression             870 non-null int64
direction_de_vent    870 non-null int64
vitesse_de_vent      870 non-null float64
temperature          870 non-null float64
humidité             870 non-null int64
visibilité           870 non-null int64
nebulosite           870 non-null int64
hauteur_neige        870 non-null float64
precipitations_1     870 non-null float64
precipitations_3     870 non-null float64
nom                  870 non-null object
latitude             870 non-null float64
longitude            870 non-null float64
timestamp            870 non-null object
dtypes: float64(7), int64(6), object(2)
memory usage: 102.1+ KB


In [69]:
df.columns

Index(['numero', 'pression', 'direction_de_vent', 'vitesse_de_vent',
       'temperature', 'humidité', 'visibilité', 'nebulosite', 'hauteur_neige',
       'precipitations_1', 'precipitations_3', 'nom', 'latitude', 'longitude',
       'timestamp', 'rr'],
      dtype='object')